## Data Scraping of initial formation page

In [2]:
import requests
from bs4 import BeautifulSoup
import csv

In [3]:
def get_table_infos(table_code) :
    all_rows = table_code.find_all("tr")

    all_semesters = {}
    for row in all_rows :
        semester_nbr = row.find("th").text.strip()
        semester_moduls = row.find("td").text.strip().split("•")
        all_semesters[semester_nbr] = semester_moduls

    return all_semesters

In [4]:
def get_master_infos(url) :
    result = requests.get(url)

    soup = BeautifulSoup(result.content, "lxml")

    mst_objectif = soup.find(id="objectifs--tab").text.strip()
    mst_program = get_table_infos(soup.find(id="programme-tab"))
    mst_skills1 = soup.find(id="competences-visees-et-debouches-tab")
    mst_skills1 = mst_skills1.find_all("li")
    mst_skills = []
    for skill in mst_skills1 :
        mst_skills.append(skill.text.strip())
    mst_Coord = soup.find(id="coordinateur-tab").text.strip()

    # print(mst_objectif,"\n" , mst_program, "\n" ,mst_skills, "\n" ,mst_Coord)

    return mst_objectif, mst_program, mst_skills, mst_Coord

# get_master_infos("https://fstt.ac.ma/Portail2023/biologie-chimie-geologie/")
# get_master_infos("https://fstt.ac.ma/Portail2023/lst-analytique-des-donnees/")
# get_master_infos("https://fstt.ac.ma/Portail2023/mst-intelligence-artificielle-et-sciences-de-donnees/")    

In [5]:

formation_initial = {
    "mst" : "https://fstt.ac.ma/Portail2023/cycle-master/",
    "lst" : "https://fstt.ac.ma/Portail2023/cycle-licence/",
    "deust" : "https://fstt.ac.ma/Portail2023/deust/",
}

formation_initial_infos = []

i = 0

for key in formation_initial.keys() :
    formation_type = key
    url = formation_initial[key]
    print("*"*50)
    print(f"formation_type : {formation_type}  => url : {url}")

    result = requests.get(url)

    soup = BeautifulSoup(result.content, "lxml")

    formation_container = soup.find("div", {"class": "elementor-element elementor-element-78ced8b elementor-widget elementor-widget-theme-post-content"})
    formation_container = formation_container.find("div", {"class": "elementor-section-wrap"})
    all_formations = formation_container.find_all("section")

    print(f"length of {formation_type} => {len(all_formations)}")

    for formation in all_formations:

        mst_name = formation.find("span", {"class": "elementor-icon-list-text"})

        if mst_name == None :
            mst_name = formation.find("h4")

        if mst_name != None :
            mst_name = mst_name.text.strip()
        else : continue

        mst_link = formation.find("div", {"class": "elementor-row"}).contents[1].attrs["data-wts-url"]
        mst_link2 = formation.find("a")

        if mst_link2 != None :
            mst_link = mst_link2.attrs["href"]

        mst_objectif, mst_program, mst_skills, mst_Coord = get_master_infos(mst_link)

        
        mst_skills = " | ".join(mst_skills)
        mst_program2 = []
        for key in mst_program.keys() :
            # print(mst_program[key])
            value = " | ".join(mst_program[key])
            mst_program2.append(f"{key} : {value[4:]}")
        
        i += 1
        formation_initial_infos.append({
            "No": i,
            "mst_name": mst_name,
            "formation_type" : formation_type,
            "mst_link": mst_link,
            "mst_objectif": mst_objectif,
            "mst_program": mst_program,
            "mst_skills": mst_skills,
            "mst_Coord": mst_Coord,
        })

        
        # print("*"*50)
        # print(f"MST Name : {mst_name}  => mst_link : {mst_link}")
        print(f"{i} - MST Name : {mst_name} ")
        # print(mst_objectif[:30],"\n" , mst_program2, "\n" ,mst_skills, "\n" ,mst_Coord)

**************************************************
formation_type : mst  => url : https://fstt.ac.ma/Portail2023/cycle-master/
length of mst => 18
1 - MST Name : MST : Analyse Appliquée et Ingénierie Statistique 
2 - MST Name : MST : Bases Cellulaires et Moléculaires en Biotechnologie 
3 - MST Name : MST : Environnement, Aquaculture et Développement Durable 
4 - MST Name : MST : Géoressources Energétiques et Réservoirs 
5 - MST Name : MST : Génie Civil 
6 - MST Name : MST : Génie des Matériaux pour Plasturgie et Métallurgie 
7 - MST Name : MST : Génie Energétique 
8 - MST Name : MST : Ingénierie Environnementale, Changement Climatique et Développement Durable 
9 - MST Name : MST : Intelligence Artificielle et Sciences de Données 
10 - MST Name : MST : Mobiquité et Big Data 
11 - MST Name : MST : Modélisation Mathématique et Science de Données 
12 - MST Name : MST :  Sciences Agroalimentaires 
13 - MST Name : MST : Sciences de l'Environnement 
14 - MST Name : MST : Sciences du Littoral:

In [6]:
columns = formation_initial_infos[0].keys()

with open("fstt-formation-initial.csv", "w", encoding="utf-8-sig", newline="") as csv_file:
    dict_writer = csv.DictWriter(csv_file, columns)
    dict_writer.writeheader()
    dict_writer.writerows(formation_initial_infos)

    print("*"*50, "\n file created \n", "*"*50)

************************************************** 
 file created 
 **************************************************
